In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_full_pl_table(stock_code):
    url = f"https://www.screener.in/company/{stock_code}/consolidated/"
    headers = {"User-Agent": "Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    # Find Profit & Loss table
    pl_table = None
    for table in soup.find_all("table"):
        if table.find_previous("h2") and "Profit & Loss" in table.find_previous("h2").text:
            pl_table = table
            break

    if not pl_table:
        print("Profit & Loss table not found.")
        return None

    rows = pl_table.find_all("tr")
    data = []

    for row in rows:
        cols = row.find_all(["th", "td"])
        cols = [col.text.strip() for col in cols]
        data.append(cols)

    # First row is header (e.g., ['', 'Mar 20', 'Mar 21', ...])
    header = data[0]
    df = pd.DataFrame(data[1:], columns=header)

    # Optional: set first column (line items) as index
    df.set_index(df.columns[0], inplace=True)

    return df

import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_shareholding_pattern(stock_code):
    url = f"https://www.screener.in/company/{stock_code}/share-holding/"
    headers = {"User-Agent": "Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    # Look for heading "Shareholding Pattern"
    heading = soup.find(lambda tag: tag.name in ["h2", "h3"] and "Shareholding Pattern" in tag.text)
    if not heading:
        print(f"⚠️ 'Shareholding Pattern' heading not found for {stock_code}")
        return None

    # Find the table just after that heading
    table = heading.find_next("table")
    if not table:
        print(f"⚠️ Shareholding table not found under heading for {stock_code}")
        return None

    # Extract headers
    headers = [th.text.strip() for th in table.find("thead").find_all("th")]

    # Extract rows
    rows = []
    for row in table.find("tbody").find_all("tr"):
        cols = [td.text.strip().replace("\n", "") for td in row.find_all("td")]
        rows.append(cols)

    # Build DataFrame
    df = pd.DataFrame(rows, columns=headers)
    df.set_index(headers[0], inplace=True)  # Set 'Quarter' or 'Date' as index
    return df


def get_cash_flow_table(stock_code):
    url = f"https://www.screener.in/company/{stock_code}/consolidated/"
    headers = {"User-Agent": "Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    cash_flow_table = None
    for table in soup.find_all("table"):
        if table.find_previous("h2") and "Cash Flow" in table.find_previous("h2").text:
            cash_flow_table = table
            break

    if not cash_flow_table:
        print("Cash Flow table not found.")
        return None

    rows = cash_flow_table.find_all("tr")
    data = []

    for row in rows:
        cols = row.find_all(["th", "td"])
        cols = [col.text.strip() for col in cols]
        data.append(cols)

    header = data[0]
    df = pd.DataFrame(data[1:], columns=header)
    df.set_index(df.columns[0], inplace=True)
    return df


def get_ratios_table(stock_code):
    url = f"https://www.screener.in/company/{stock_code}/consolidated/"
    headers = {"User-Agent": "Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    ratios_table = None
    for table in soup.find_all("table"):
        if table.find_previous("h2") and "Ratios" in table.find_previous("h2").text:
            ratios_table = table
            break

    if not ratios_table:
        print("Ratios table not found.")
        return None

    rows = ratios_table.find_all("tr")
    data = []

    for row in rows:
        cols = row.find_all(["th", "td"])
        cols = [col.text.strip() for col in cols]
        data.append(cols)

    header = data[0]
    df = pd.DataFrame(data[1:], columns=header)
    df.set_index(df.columns[0], inplace=True)
    return df





                   Mar 2014 Mar 2015 Mar 2016 Mar 2017 Mar 2018 Mar 2019  \
                                                                           
Revenue +            42,555   50,666   63,162   73,271   85,288  105,161   
Interest             23,445   27,288   34,070   38,042   42,381   53,713   
Expenses +           13,508   16,164   20,055   23,856   29,532   34,856   
Financing Profit      5,602    7,214    9,037   11,374   13,374   16,592   
Financing Margin %      13%      14%      14%      16%      16%      16%   

                   Mar 2020 Mar 2021 Mar 2022 Mar 2023 Mar 2024 Mar 2025  
                                                                          
Revenue +           122,189  128,552  135,936  170,754  283,649  336,367  
Interest             62,137   59,248   58,584   77,780  154,139  183,894  
Expenses +           45,459   52,457   56,557   63,042  174,196  190,780  
Financing Profit     14,593   16,848   20,795   29,932  -44,685  -38,306  
Financing Margin 

In [11]:
import pandas as pd

def process_stock(stock):
    print(f"Processing: {stock}")
    
    # Get all data tables
    pl = get_full_pl_table(stock)
    ratios = get_ratios_table(stock)
    cash = get_cash_flow_table(stock)
    # share = get_shareholding_pattern(stock)

    # Standardize indexes
    pl.index.name = "KPI"
    ratios.index.name = "KPI"
    cash.index.name = "KPI"
    # share.index.name = "Quarter"

    # Transpose and rename
    df_pl = pl.transpose().reset_index().rename(columns={"index": "Quarter"})
    df_ratios = ratios.transpose().reset_index().rename(columns={"index": "Quarter"})
    df_cash = cash.transpose().reset_index().rename(columns={"index": "Quarter"})
    # df_share = share.reset_index().rename(columns={share.index.name: "Quarter"})

    # Merge all tables on Quarter
    merged = df_pl.merge(df_ratios, on="Quarter", how="outer")
    merged = merged.merge(df_cash, on="Quarter", how="outer")
    # merged = merged.merge(df_share, on="Quarter", how="outer")

    # Add stock name
    merged["Stock"] = stock

    # Move Stock and Quarter to front
    cols = ["Stock", "Quarter"] + [col for col in merged.columns if col not in ["Stock", "Quarter"]]
    merged = merged[cols]

    return merged


# --- Process for all stocks ---
stock_list = ["HDFCBANK", "ICICIBANK"]

all_data = pd.DataFrame()

for stock in stock_list:
    try:
        df = process_stock(stock)
        all_data = pd.concat([all_data, df], ignore_index=True)
    except Exception as e:
        print(f"Error with {stock}: {e}")

# 💾 Save to CSV
all_data.to_csv("final_stock_kpis_auto.csv", index=False)

# 🖨 Preview
print("\n✅ Final Combined Table (Auto KPIs):\n")
print(all_data)


Processing: HDFCBANK
Processing: ICICIBANK

✅ Final Combined Table (Auto KPIs):

KPI      Stock   Quarter Revenue + Interest Expenses + Financing Profit  \
0     HDFCBANK  Mar 2014    42,555   23,445     13,508            5,602   
1     HDFCBANK  Mar 2015    50,666   27,288     16,164            7,214   
2     HDFCBANK  Mar 2016    63,162   34,070     20,055            9,037   
3     HDFCBANK  Mar 2017    73,271   38,042     23,856           11,374   
4     HDFCBANK  Mar 2018    85,288   42,381     29,532           13,374   
5     HDFCBANK  Mar 2019   105,161   53,713     34,856           16,592   
6     HDFCBANK  Mar 2020   122,189   62,137     45,459           14,593   
7     HDFCBANK  Mar 2021   128,552   59,248     52,457           16,848   
8     HDFCBANK  Mar 2022   135,936   58,584     56,557           20,795   
9     HDFCBANK  Mar 2023   170,754   77,780     63,042           29,932   
10    HDFCBANK  Mar 2024   283,649  154,139    174,196          -44,685   
11    HDFCBANK  Mar

In [12]:
import requests
from bs4 import BeautifulSoup

def show_all_headers(stock_code):
    url = f"https://www.screener.in/company/{stock_code}/consolidated/"
    headers = {"User-Agent": "Mozilla/5.0"}
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    header_tags = ["h1", "h2", "h3", "h4"]
    all_headers = []

    for tag in header_tags:
        for element in soup.find_all(tag):
            text = element.get_text(strip=True)
            all_headers.append((tag, text))

    # Display neatly
    for tag, text in all_headers:
        print(f"{tag.upper()}: {text}")

# Example usage
show_all_headers("HDFCBANK")


H1: HDFC Bank Ltd
H1: HDFC Bank Ltd
H2: Peer comparison
H2: Quarterly Results
H2: Profit & Loss
H2: Balance Sheet
H2: Cash Flows
H2: Ratios
H2: Shareholding Pattern
H2: Documents
H3: Announcements
H3: Annual reports
H3: Credit ratings
H3: Concalls
